### Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:
* если источник traffic_source равен yandex или google, то в source_type ставится organic
* для источников paid и email из России - ставим ad
* для источников paid и email не из России - ставим other
* все остальные варианты берем из traffic_source без изменений


In [14]:
import pandas as pd

df = pd.read_csv('visit_log.csv', ';')


def source_type(row):
    if row['traffic_source'] == 'yandex' or row['traffic_source'] == 'google':
        return 'organic'
    elif (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email'):
        if row['region'] == 'Russia':
            return 'ad'
        else:
            return 'other'

df['source_type'] = df.apply(source_type, axis = 1)
df.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,None
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


### Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:
1.    Прочитайте содержимое файла с датафрейм
2.    Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном


In [27]:
import pandas as pd

df = pd.read_csv('URLs.txt')

df[df.url.str.contains(pat = '\d{8}-', regex = True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


### Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [170]:
import pandas as pd

df = pd.read_csv('ratings.csv')

df_cnt = df[['userId', 'movieId']].groupby(['userId']).count().reset_index()
df_cnt.rename(columns = {'movieId':'cnt'}, inplace = True) 
df = df[['userId', 'timestamp']].merge(df_cnt, how='left', on='userId')
df = df[df['cnt'] > 100]
df = df.groupby('userId').agg([min, max])
df['diff'] = (df['timestamp']['max'] - df['timestamp']['min'])
df['diff'].mean()/3600/24

463.89476214829165

### Задание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента


In [207]:
import pandas as pd

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

df = pd.concat([rzd, auto, air])
df.fillna(0.0, inplace=True)
df = df.groupby(['client_id']).agg({'rzd_revenue':'sum', 'auto_revenue':'sum', 'air_revenue':'sum'}).reset_index()
df.head()

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0


In [208]:
df = df.merge(client_base, how='left', on='client_id')
df.head()

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
